In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import os
import sys

sys.path.insert(0, os.path.abspath(".."))
from dotenv import load_dotenv
import seaborn as sns
import pandas as pd
import numpy as np

from db import PostgresDB
from schemas import Proband
from cohorts import Cohort, EncounterComparator

In [18]:
load_dotenv()

True

In [19]:
db = PostgresDB(
    db_name=os.getenv("DB_NAME"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    user=os.getenv("DB_USER"),
)


Connected to database


In [20]:
cohort = Cohort(db=db)

In [21]:
cohort.get_sepsis_cohort(size=10)

In [22]:
cohort.initialize_data()

In [23]:
r = cohort.compare_encounters(
    demographics_weight=0.0,
    diagnoses_weight=0.0,
    labevents_weight=0.0,
    vitalsigns_weight=0.0,
    inputevents_weight=1.0,
    normalize_categories=True,
    aggregate_method="mean",
)


Finished encounter 26184834
Finished encounter 23581541
Finished encounter 20345487
Finished encounter 23822395
Finished encounter 28994087
Finished encounter 22725460
Finished encounter 20321825
Finished encounter 23473524
Finished encounter 28662225
Finished encounter 21329021
Normalizing categories by scaling to range 0..1


In [24]:
r

[{'encounter_a': 26184834, 'encounter_b': 26184834, 'similarity': 1},
 {'encounter_a': 26184834,
  'encounter_b': 23581541,
  'similarity': 0.19130434782608696},
 {'encounter_a': 26184834,
  'encounter_b': 20345487,
  'similarity': 0.29399585921325055},
 {'encounter_a': 26184834,
  'encounter_b': 23822395,
  'similarity': 0.2864450127877239},
 {'encounter_a': 26184834, 'encounter_b': 28994087, 'similarity': 0.0},
 {'encounter_a': 26184834,
  'encounter_b': 22725460,
  'similarity': 0.21376811594202902},
 {'encounter_a': 26184834,
  'encounter_b': 20321825,
  'similarity': 0.43840579710144933},
 {'encounter_a': 26184834,
  'encounter_b': 23473524,
  'similarity': 0.469038208168643},
 {'encounter_a': 26184834,
  'encounter_b': 28662225,
  'similarity': 0.5683432793407218},
 {'encounter_a': 26184834,
  'encounter_b': 21329021,
  'similarity': 0.07502131287297528},
 {'encounter_a': 23581541,
  'encounter_b': 26184834,
  'similarity': 0.19130434782608696},
 {'encounter_a': 23581541, 'encoun

In [25]:
cohort.similarity_encounters[0].vitalsigns

[Vitalsign(id='heart_rate', value=70.8, subject_id=10001884, hadm_id=26184834, name='heart_rate'),
 Vitalsign(id='sbp_ni', value=129.1764705882353, subject_id=10001884, hadm_id=26184834, name='sbp_ni'),
 Vitalsign(id='dbp_ni', value=74.66968325791855, subject_id=10001884, hadm_id=26184834, name='dbp_ni'),
 Vitalsign(id='mbp_ni', value=90.52941176470588, subject_id=10001884, hadm_id=26184834, name='mbp_ni'),
 Vitalsign(id='resp_rate', value=19.0, subject_id=10001884, hadm_id=26184834, name='resp_rate'),
 Vitalsign(id='temperature', value=37.01076923076923, subject_id=10001884, hadm_id=26184834, name='temperature'),
 Vitalsign(id='spo2', value=97.64864864864865, subject_id=10001884, hadm_id=26184834, name='spo2'),
 Vitalsign(id='glucose', value=149.33333333333334, subject_id=10001884, hadm_id=26184834, name='glucose')]

In [26]:
cohort.similarity_encounters[1].vitalsigns

[Vitalsign(id='heart_rate', value=94.21428571428571, subject_id=10002013, hadm_id=23581541, name='heart_rate'),
 Vitalsign(id='sbp_ni', value=110.5, subject_id=10002013, hadm_id=23581541, name='sbp_ni'),
 Vitalsign(id='dbp_ni', value=64.5, subject_id=10002013, hadm_id=23581541, name='dbp_ni'),
 Vitalsign(id='mbp_ni', value=75.4, subject_id=10002013, hadm_id=23581541, name='mbp_ni'),
 Vitalsign(id='resp_rate', value=15.178571428571429, subject_id=10002013, hadm_id=23581541, name='resp_rate'),
 Vitalsign(id='temperature', value=37.20444444444444, subject_id=10002013, hadm_id=23581541, name='temperature'),
 Vitalsign(id='spo2', value=96.73076923076923, subject_id=10002013, hadm_id=23581541, name='spo2'),
 Vitalsign(id='glucose', value=149.4090909090909, subject_id=10002013, hadm_id=23581541, name='glucose')]

In [27]:
from vitalsigns import VitalsignComparator

vs_comp = VitalsignComparator(db=db)
vs_comp.compare(
    cohort.similarity_encounters[0].vitalsigns,
    cohort.similarity_encounters[1].vitalsigns,
    scale_by_distribution=False,
)


0.6435590079267861

In [28]:
comp = EncounterComparator(db=db)


In [29]:
comp.compare(cohort.similarity_encounters[0], cohort.similarity_encounters[1], demographics_weight=0.0, labevents_weight=0.9, diagnoses_weight=0.1, aggregate_method="mean")

0.23337146705810025